In [126]:
import torchvision.transforms as transforms
import torchvision.datasets as dset
import torch
import torch.nn
import numpy as np 
import struct
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from facenet_model import *
import torch.optim as optim
import tqdm

In [ ]:
!cp /content/M

In [44]:
with open('./db_train.raw', 'rb') as f:
    data = np.fromfile(f, dtype=np.dtype(np.uint8))
    data = data.reshape((111430,56, 56, 3))
with open('./label_train.txt', 'rb') as f:
    label=f.read().splitlines()
    for k,elem in enumerate(label):
        label[k]=int(elem)
    label=np.array(label)

In [46]:
class dataset(Dataset):
    def __init__(self, data_0, data_1):
        self.samples = []
        self.transform=transforms.Compose([  transforms.RandomHorizontalFlip(p=0.5),
                                transforms.RandomAffine(20,(0.12,0.12),(0.8,1.2),interpolation=transforms.InterpolationMode.NEAREST,fill=0),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                ])
        for img in data_0:
            img=transforms.ToTensor()(img)
            self.samples.append((img,0))
        for img in data_1:
            img=transforms.ToTensor()(img)
            self.samples.append((img,1))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, id):
        (img,label)=self.samples[id]
        img=self.transform(img)
        return (img,label)

In [48]:
data_0=data[np.where(label==0)[0]]
data_1=data[np.where(label==1)[0]]
data_0_t=data_0[:int(len(data_0)*0.7)]
data_1_t=data_1[:int(len(data_1)*0.7)]
data_0_v=data_0[int(len(data_0)*0.7):]
data_1_v=data_1[int(len(data_1)*0.7):]

In [49]:
train_ds=dataset(data_0_t,data_1_t)
val_ds=dataset(data_0_v,data_1_v)

In [50]:
dataloader_t = torch.utils.data.DataLoader(train_ds,batch_size=32,shuffle=True,drop_last=True)
dataloader_v = torch.utils.data.DataLoader(val_ds,batch_size=32,shuffle=True,drop_last=True)

# Model

In [127]:
device= torch.device("cuda:0")

In [128]:
facenet=FaceNet(0.2,0.7,True).to(device)

AssertionError: Torch not compiled with CUDA enabled

In [56]:
optimizer = optim.Adam(facenet.parameters(), lr=3e-4, betas=(0.9, 0.999))
alpha1=0.2
alpha2=0.2
num_epochs=7

In [119]:
def lossfunction(x,y):
    num_0=torch.where(y==0)[0].shape[0]
    num_1=x.shape[0]-num_0
    w0=0.
    w1=0.
    if num_0!=0:
        w0=torch.divide(x.shape[0],(2*num_0),).float()
    if num_1!=0:
        w1=torch.divide(x.shape[0],(2*num_1),).float()
    w=torch.ones_like(x).float()
    w[torch.where(y==0)[0]]=w0
    w[torch.where(y==1)[0]]=w1
    return torch.nn.BCELoss(w)(x.float(),y.float())

In [116]:
Loss_train=[]
Loss_val=[]
for epoch in tqdm.tqdm(range(num_epochs)):
    L_t=[]
    L_v=[]
    facenet.train()
    for i, dataj in enumerate(dataloader_t, 0):
        facenet.zero_grad()
        x=dataj[0].float().to(device)
        yh=dataj[1].float().to(device)
        y,aux1,aux2=facenet(x)
        loss=lossfunction(x,yh)
        loss_aux1=lossfunction(aux1,yh)
        loss_aux2=lossfunction(aux2,yh)
        total_loss=(loss+alpha1*loss_aux1+alpha2*loss_aux2)/(1+alpha1+alpha2)
        total_loss.backward()
        optimizer.step()
        L_t.append([loss.item(),loss_aux1.item(),loss_aux2.item(),total_loss.item()])
    facenet.eval()
    for i, dataj in enumerate(dataloader_v, 0):
        x=dataj[0].float().to(device)
        yh=dataj[1].float().to(device)
        y,aux1,aux2=facenet(x)
        loss=lossfunction(x,yh)
        L_v.append(loss)
    err_t=np.mean(L_t,0)
    err_v=np.mean(L_v,0)
    Loss_train.append(err_t)
    Loss_val.append(err_v)
    print("Erreur Training {} \t Erreur Val {}".format(err_t,err_v))